In [5]:
import gym
from gym import spaces
import numpy as np
import os
import cv2

# Define the custom Gym environment for weapon detection
class WeaponDetectionEnv(gym.Env):
    def __init__(self, dataset_path):
        super(WeaponDetectionEnv, self).__init__()

        # Load images from the dataset
        self.image_data = self.load_images(dataset_path)
        self.num_images = len(self.image_data)

        # Define action and observation spaces
        self.action_space = spaces.Discrete(2)  # 0: No weapon, 1: Weapon detected
        self.observation_space = spaces.Box(low=0, high=255, shape=(64, 64, 3), dtype=np.uint8)  # RGB image

        # Initialize any other environment-specific variables and parameters here
        self.current_image_index = 0
        self.weapon_present = False  # True if a weapon is present in the image

    def load_images(self, dataset_path):
        image_data = []
        for filename in os.listdir(dataset_path):
            if filename.endswith(".jpg"):  # Assuming the images are in JPEG format
                image_path = os.path.join(dataset_path, filename)
                image = cv2.imread(image_path)
                if image is not None:
                    image = cv2.resize(image, (64, 64))
                    image_data.append(image)
                else:
                    print(f"Failed to load image: {image_path}")
        return image_data

    def step(self, action):
        # Simulate an action
        self.weapon_present = (action == 1)

        # Calculate reward
        if self.weapon_present:
            reward = 1.0  # Positive reward for correctly detecting a weapon
        else:
            reward = 0.0  # No reward for not detecting a weapon

        self.current_image_index = (self.current_image_index + 1) % self.num_images
        current_image = self.image_data[self.current_image_index]

        done = (self.current_image_index == 0)  # End episode after cycling through all images
        return current_image, reward, done, {}

    def reset(self):
        # Reset the environment for a new episode

        if not self.image_data:
            raise ValueError("The dataset is empty or exhausted. Make sure it contains images.")
        
        # Ensure the index stays within the valid range
        self.current_image_index = 0

        # Load the current image
        current_image = self.image_data[self.current_image_index]

        self.weapon_present = np.random.choice([True, False])  # Simulated presence of a weapon

        # Increment the index for the next episode
        self.current_image_index += 1

        return current_image

# Instantiate the environment with the dataset path
dataset_path =r"C:\NewML\MachineLearning\env\archive\weapon_detection\val\images\no_weapon"
env = WeaponDetectionEnv(dataset_path)

# Training loop
total_episodes = 1000
successful_episodes = 0

for episode in range(total_episodes):
    state = env.reset()

    for _ in range(1):  # In this simplified example, just take one action per episode
        # Replace this with your RL agent's action selection logic
        action = np.random.choice([0, 1])  # Random action
        next_state, reward, done, _ = env.step(action)

        if reward > 0:  # Agent successfully detected the weapon
            successful_episodes += 1

# Calculate accuracy
accuracy = successful_episodes / total_episodes * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 50.60%
